In [ ]:
!pip install --upgrade git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-unxb6_ec
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-unxb6_ec
  Resolved https://github.com/huggingface/transformers to commit 36c9181f5cc748b174f02f133ff17376e3969e4c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import re
from time import time

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation",
                     model="unsloth/Llama-3.2-3B-Instruct",
                     max_new_tokens = 128,
                     repetition_penalty = 1.1)

Device set to use cuda:0


In [ ]:
example_queries = [
    "최근 AI 업계에서 비디오 생성 모델이 뭐가 있을까? 그리고 그 모델들의 성능 비교는 어떻게 되고, 앞으로의 전망은 어때?",
    "CES2025의 미래 기술 트렌드에 대해 요약해줘",
    "AI 윤리 문제에 대해서 앞으로 어떤 변화가 있을 것 같아?"
]

In [ ]:
prompt ="""
You are an intelligent assistant specializing in generating search-friendly keyword phrases.
Your task is to extract concise, contextually accurate keyword phrases from a given user query.
These keyword phrases should be optimized for news search and adhere to the following guidelines:

1. Ensure each phrase is specific and clear.
2. Maintain the user's intended context.
3. Include relevant dates, time periods, or locations when applicable.
4. Use Named Entities (e.g., people, places, organizations, events) mentioned in the query.
5. Incorporate synonyms or related terms to broaden the search scope.
6. Keep phrases concise and avoid redundant words.
7. Construct keyword phrases suitable for Boolean operators (e.g., AND, OR).
8. Include regional or contextual relevance when specified.
9. Reflect actions or dynamic processes, not just static terms.
10. Ensure the phrases are structured for search engine optimization.

**Output Format**:
- Provide 3-5 keyword phrases as a numbered list.
- Each phrase should be a complete and logical search query.

**Example User Query**:
“2025년에 헬스케어 분야에서 AI 기술이 어떻게 발전하고 있나요?”

**Example Output**:
1. 2025년 AI 기술 헬스케어 발전
2. AI 헬스케어 트렌드 2025
3. 2025년 헬스케어 AI 최신 동향
4. AI 기반 헬스케어 혁신 2025

Now, extract keyword phrases from the following query:
"{USER_QUERY}"
"""

In [ ]:
response = generator(prompt.format(USER_QUERY = example_queries[0]))[0]['generated_text']
pattern = r"^\d+\.\s(.+)"
key_phrase = re.findall(pattern, response, re.MULTILINE)
key_phrase = key_phrase[15:]

key_phrase

['최근 AI 비디오 생성 모델의 성능 분석',
 'AI 비디오 생성 모델의 향후 전망',
 '비디오 생성 모델의 AI 기술 발전 상황',
 'AI 기반 비디오 생성 모델 성능 비교']

In [ ]:
def extract_key_phrase(generator, query):
    prompt ="""
You are an intelligent assistant specializing in generating search-friendly keyword phrases.
Your task is to extract concise, contextually accurate keyword phrases from a given user query.
These keyword phrases should be optimized for news search and adhere to the following guidelines:

1. Ensure each phrase is specific and clear.
2. Maintain the user's intended context.
3. Include relevant dates, time periods, or locations when applicable.
4. Use Named Entities (e.g., people, places, organizations, events) mentioned in the query.
5. Incorporate synonyms or related terms to broaden the search scope.
6. Keep phrases concise and avoid redundant words.
7. Construct keyword phrases suitable for Boolean operators (e.g., AND, OR).
8. Include regional or contextual relevance when specified.
9. Reflect actions or dynamic processes, not just static terms.
10. Ensure the phrases are structured for search engine optimization.

**Output Format**:
- Provide 3-5 keyword phrases as a numbered list.
- Each phrase should be a complete and logical search query.

**Example User Query**:
“2025년에 헬스케어 분야에서 AI 기술이 어떻게 발전하고 있나요?”

**Example Output**:
1. 2025년 AI 기술 헬스케어 발전
2. AI 헬스케어 트렌드 2025
3. 2025년 헬스케어 AI 최신 동향
4. AI 기반 헬스케어 혁신 2025

Now, extract keyword phrases from the following query:
"{USER_QUERY}"
"""
    final_prompt = prompt.format(USER_QUERY = query)
    response = generator(final_prompt)[0]['generated_text']

    # response -> phrase list 변환
    pattern = r"^\d+\.\s(.+)"
    key_phrase = re.findall(pattern, response, re.MULTILINE)
    key_phrase = key_phrase[15:]

    return key_phrase

In [ ]:
key_phrases = extract_key_phrase(generator, example_queries[0])

['최근 비디오 생성 모델 트렌드',
 '비디오 생성 모델의 미래 전망',
 'AI 기반 비디오 생성 모델 비교',
 '비디오 생성 모델 성능 향상 방법']

In [ ]:
def extract_key_words(generator, query):
    prompt = """
You are an intelligent assistant specializing in keyword extraction.
Your task is to extract **Matched Keywords** and **Related Keywords** from a given document.
The output should be presented as two separate numbered lists for clarity.
Make sure that the list should be written in query language.

**Guidelines**:
1. **Matched Keywords**: Extract up to 5 keywords explicitly mentioned in the document.
2. **Related Keywords**: Extract up to 5 keywords contextually related to the document but not explicitly mentioned.

**Output Format**:
- Matched Keywords:
1. [Keyword 1]
2. [Keyword 2]
3. [Keyword 3]
...
- Related Keywords:
1. [Keyword 1]
2. [Keyword 2]
3. [Keyword 3]
...

**Example Input**:
Document: "트랜스포머 기법을 쓰지 않은 LLM모델의 트렌드"

**Example Output**:
- Matched Keywords:
1. 트랜스포머
2. 기법
3. LLM
4. 모델

- Related Keywords:
1. 자연어
2. 언어모델
3. AI
4. 챗봇

Now, extract keywords for the following document:
Document: {USER_QUERY}
"""
    prompt_final = prompt.format(USER_QUERY = query)
    response = generator(prompt_final)[0]['generated_text']

    key_words = re.findall(pattern, response, re.MULTILINE)
    key_words = key_words[16:]
    return key_words


In [ ]:
key_phrases = extract_key_phrase(generator, example_queries[0])
key_phrases

['AI 비디오 생성 모델 전망',
 '최근 비디오 생성 모델 성능',
 'AI 비디오 생성 모델 최신 동향',
 '비디오 생성 모델 성능 향상 전망 ']

In [ ]:
key_words = extract_key_words(generator, example_queries[0])
key_words

['비디오', '생성', '모델', '성능', '전망', 'AI', '컴퓨터', '화면', '영상', '인공지능']

In [ ]:
search_keywords = key_phrases + key_words
search_keywords

['AI 비디오 생성 모델 전망',
 '최근 비디오 생성 모델 성능',
 'AI 비디오 생성 모델 최신 동향',
 '비디오 생성 모델 성능 향상 전망 ',
 '비디오',
 '생성',
 '모델',
 '성능',
 '전망',
 'AI',
 '컴퓨터',
 '화면',
 '영상',
 '인공지능']